In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as math
from IPython.display import display
from IPython.core.pylabtools import getfigs
from sklearn import linear_model 
import time
import pymysql as mdb
import re
#import seaborn as sns

In [2]:
%matplotlib qt

In [3]:
from prep_predictors_for_ml import *
%load_ext autoreload
%autoreload 2

In [5]:
con = mdb.connect('localhost','root','hobbes','kickstarter')

In [18]:
with con:
    cur = con.cursor()
    cur.execute("DESCRIBE Predictors")
    rows = cur.fetchall()
    cur.close()

In [19]:
col_names = np.array([row[0] for row in rows])
print col_names

['Id' 'Url' 'Outcome' 'Pledged' 'Goal' 'Category' 'Subcategory' 'Currency'
 'Country' 'Location' 'Launch_s' 'Campaign_days' 'Title_n_words'
 'Blurb_n_words' 'Year' 'Month' 'Month_day' 'Week_day' 'Hour' 'Has_video'
 'N_rewards' 'Min_reward' 'Median_reward' 'Max_reward' 'Reward_ship_days'
 'Ships_intn' 'N_links' 'N_figs' 'Body_n_words' 'N_creator_backed'
 'N_creator_created']


In [5]:
# in sql query, specify: 
# launch_t > 0, 
# n_rewards > -1, 
# ship_date > -1
# launch t >= 1331091810 (March 6, 2012 - earliest intn ship date)
# status != live

In [20]:
with con:
    cur = con.cursor()
    cur.execute("SELECT * FROM Predictors WHERE \
                Launch_s > 1331091810 AND Reward_ship_days >= 0 AND N_creator_backed > -1 \
                AND N_rewards > -1 AND Outcome != 'live' \
                AND Country = 'US' AND Currency = 'USD'")
    predictors_rows = cur.fetchall()
    cur.close()

In [6]:
# run sql query, convert to df

In [25]:
new_predictors_df = pd.DataFrame(np.asarray(predictors_rows),\
                                 columns=col_names)

In [26]:
new_predictors_df.tail(2)

,Id,Url,Outcome,Pledged,Goal,Category,Subcategory,Currency,Country,Location,...,Min_reward,Median_reward,Max_reward,Reward_ship_days,Ships_intn,N_links,N_figs,Body_n_words,N_creator_backed,N_creator_created
115194,191382,https://www.kickstarter.com/projects/209708895...,suspended,0,31,art,Painting,USD,US,lakeland-fl,...,8,31,1001,111,0,2,0,504,0,1
115195,191383,https://www.kickstarter.com/projects/thelifecu...,suspended,10084,15000,art,Public Art,USD,US,black-rock-city-nv,...,10,200,1000,155,1,1,7,1246,130,2


In [28]:
new_predictors_df.to_pickle('june24_predictors_df.save')

In [29]:
print col_names

['Id' 'Url' 'Outcome' 'Pledged' 'Goal' 'Category' 'Subcategory' 'Currency'
 'Country' 'Location' 'Launch_s' 'Campaign_days' 'Title_n_words'
 'Blurb_n_words' 'Year' 'Month' 'Month_day' 'Week_day' 'Hour' 'Has_video'
 'N_rewards' 'Min_reward' 'Median_reward' 'Max_reward' 'Reward_ship_days'
 'Ships_intn' 'N_links' 'N_figs' 'Body_n_words' 'N_creator_backed'
 'N_creator_created']


In [41]:
# to include 
X_cols = ['Goal','Campaign_days','Title_n_words','Blurb_n_words',
          'Has_video','N_rewards','Min_reward',
          'Median_reward','Max_reward','Reward_ship_days','N_links',
          'N_figs','Body_n_words','N_creator_backed','N_creator_created']

In [42]:
new_X_df = new_predictors_df.ix[:,X_cols]

In [43]:
new_X_df.head(3)

,Goal,Campaign_days,Title_n_words,Blurb_n_words,Has_video,N_rewards,Min_reward,Median_reward,Max_reward,Reward_ship_days,N_links,N_figs,Body_n_words,N_creator_backed,N_creator_created
0,5000,30,2,17,1,8,10,175,5000,110,1,0,812,18,4
1,10000,30,8,21,1,10,5,125,5000,31,5,1,792,7,1
2,18000,27,5,14,1,19,5,500,10000,4,0,0,540,3,2


In [45]:
outcomes_df = make_success_fail_df(new_predictors_df.ix[:,['Pledged','Goal']])
outcomes_df.head(2)

,Outcome
0,1
1,1


In [46]:
new_ships_intn_df = make_ships_intn_df(new_predictors_df)
new_ships_intn_df.head(2)

,Ships_intn
0,0
1,0


In [47]:
categories_dummy_df = make_categorical_df_from_str(new_predictors_df['Category'])
categories_dummy_df.head(2)

,art,comics,crafts,dance,design,fashion,film,food,games,journalism,music,photography,publishing,technology,theater
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [49]:
weekday_df = make_categorical_df_from_num(new_predictors_df.loc[:,['Week_day']],\
                 col_names=['Mon','Tue','Wed','Thu','Fri','Sat','Sun'])
weekday_df.tail(2)

,Mon,Tue,Wed,Thu,Fri,Sat,Sun
115194,1,0,0,0,0,0,0
115195,0,0,0,1,0,0,0


In [50]:
# hour_df = make_categorical_df_from_num(predictor_df.loc[:,['Hour']],append_str='h')
# hour_df.tail(3)

In [51]:
# month_day_df = make_categorical_df_from_num(predictor_df.loc[:,['Monday_day']],col_names=range(1,32),append_str='d') # fix this misspelling
# month_day_df.tail(2)

In [52]:
# month_df = make_categorical_df_from_num(predictor_df.loc[:,['Month']],\
#          col_names=['Jan','Feb','Mar','Apr','May','Jun','Jul',\
#                     'Aug','Sep','Oct','Nov','Dec'])
# month_df.tail(2)

In [53]:
X_df = pd.concat([new_X_df,categories_dummy_df, 
                  new_ships_intn_df,weekday_df],axis=1)

In [54]:
X_df.head(2)

,Goal,Campaign_days,Title_n_words,Blurb_n_words,Has_video,N_rewards,Min_reward,Median_reward,Max_reward,Reward_ship_days,...,technology,theater,Ships_intn,Mon,Tue,Wed,Thu,Fri,Sat,Sun
0,5000,30,2,17,1,8,10,175,5000,110,...,0,1,0,0,0,0,0,0,0,1
1,10000,30,8,21,1,10,5,125,5000,31,...,0,1,0,0,0,0,0,0,1,0


In [55]:
X_df.to_pickle('new_X_df.save')

In [56]:
outcomes_df.to_pickle('new_outcomes_df.save')